In [74]:
library('dplyr')
library('effsize')
library('ggplot2')
library('foreign')
library('pscl')
library('ggrepel')
library('epitools')

In [2]:
dfMDD = read.csv('../Data/dfMDDMDDCurrPrin_satisfaction_rdq_dx_demo.csv')
log = read.spss('../Data/DemosDx/Tx Day Consent Log_1.sav')
nrow(dfMDD)
demo = read.spss('../Data/DemosDx/Demographics Form_1.sav')
dx = read.spss('../Data/DemosDx/Diagnosis_1.sav')

Warning message in read.spss("../Data/DemosDx/Tx Day Consent Log_1.sav"):
“Undeclared level(s) 0, 11, 12 added in variable: DC_status_1”
Warning message in read.spss("../Data/DemosDx/Tx Day Consent Log_1.sav"):
“Duplicated levels in factor DC_status_1: Other”
Warning message in read.spss("../Data/DemosDx/Tx Day Consent Log_1.sav"):
“Undeclared level(s) 2 added in variable: SCID_1”
Warning message in read.spss("../Data/DemosDx/Tx Day Consent Log_1.sav"):
“Undeclared level(s) 0 added in variable: Referral_1”


[1] 844

Warning message in read.spss("../Data/DemosDx/Demographics Form_1.sav"):
“Undeclared level(s) 6 added in variable: Race_1”
Warning message in read.spss("../Data/DemosDx/Demographics Form_1.sav"):
“Undeclared level(s) 8 added in variable: Relationship_1”
Warning message in read.spss("../Data/DemosDx/Diagnosis_1.sav"):
“../Data/DemosDx/Diagnosis_1.sav: Long string value labels record found (record type 7, subtype 21), but ignored”
Warning message in read.spss("../Data/DemosDx/Diagnosis_1.sav"):
“Undeclared level(s) 22 added in variable: eatnos_1”
Warning message in read.spss("../Data/DemosDx/Diagnosis_1.sav"):
“Undeclared level(s) 10 added in variable: addhpr_1”


In [132]:
#Reverse Scales
dfMDD$pre_cope_r = abs(dfMDD$pre_cope-2)
dfMDD$pre_pmh_r = abs(dfMDD$pre_pmh-2)
dfMDD$pre_fun_r = abs(dfMDD$pre_fun-2)
dfMDD$pre_well_r = abs(dfMDD$pre_well-2)
dfMDD$post_cope_r = abs(dfMDD$post_cope-2)
dfMDD$post_pmh_r = abs(dfMDD$post_pmh-2)
dfMDD$post_fun_r = abs(dfMDD$post_fun-2)
dfMDD$post_well_r = abs(dfMDD$post_well-2)

In [149]:
#Groups
groups = c('dep', 'any1', 'dep_plus1', 'any2', 'dep_plus2', 'any3', 'dep_plus3', 'any4', 'dep_plus4', 'any5', 'all6')

In [189]:
dep_responder = ifelse((dfMDD$pre_dsym - dfMDD$post_dsym)/(dfMDD$pre_dsym+1e-10) >= 0.5 , 1, 0)
ndep_responder = ifelse((dfMDD$pre_ndsym - dfMDD$post_ndsym)/(dfMDD$pre_ndsym+1e-10) >= 0.5, 1, 0)
cope_responder = ifelse((dfMDD$pre_cope_r - dfMDD$post_cope_r)/(dfMDD$pre_cope_r+1e-10) >= 0.5, 1, 0)
pmh_responder = ifelse((dfMDD$pre_pmh_r - dfMDD$post_pmh_r)/(dfMDD$pre_pmh_r+1e-10) >= 0.5, 1, 0)
fun_responder = ifelse((dfMDD$pre_fun_r - dfMDD$post_fun_r)/(dfMDD$pre_fun_r+1e-10) >= 0.5, 1, 0)
well_responder = ifelse((dfMDD$pre_well_r - dfMDD$post_well_r)/(dfMDD$pre_well_r+1e-10) >= 0.5, 1, 0)

nondep_domain_counts = 
     ndep_responder + cope_responder + pmh_responder + fun_responder + well_responder


any1_responder = ifelse(dep_responder + nondep_domain_counts >= 1, 1, 0)
any2_responder = ifelse(dep_responder + nondep_domain_counts >= 2, 1, 0)
any3_responder = ifelse(dep_responder + nondep_domain_counts >= 3, 1, 0)
any4_responder = ifelse(dep_responder + nondep_domain_counts >= 4, 1, 0)
any5_responder = ifelse(dep_responder + nondep_domain_counts >= 5, 1, 0)
all6_responder = ifelse(dep_responder + nondep_domain_counts >= 6, 1, 0)

dep_plus1_responder = ifelse(dep_responder == 1 & nondep_domain_counts >= 1, 1, 0)
dep_plus2_responder = ifelse(dep_responder == 1 & nondep_domain_counts >= 2, 1, 0)
dep_plus3_responder = ifelse(dep_responder == 1 & nondep_domain_counts >= 3, 1, 0)
dep_plus4_responder = ifelse(dep_responder == 1 & nondep_domain_counts >= 4, 1, 0)

In [200]:
table1 = matrix(NA, nrow = 11, ncol = 9)

for (i in 1:length(groups)){
    group = groups[i]
    responder = get(paste0(group,'_responder'))
    
    #N
    table1[i,1] = paste0(round(sum(responder)/length(responder)*100,1), '% (', sum(responder),')')
    
    #Mean (SD)
    table1[i,2] = paste0(round(mean(dfMDD$imprv1[responder==1]),1), ' (', round(sd(dfMDD$imprv1[responder==1]),1),')')
    table1[i,3] = paste0(round(mean(dfMDD$imprv1[responder==0]),1), ' (', round(sd(dfMDD$imprv1[responder==0]),2),')')
    
    
    if (sum(responder) >1){
        #T-test
        t_test_res = t.test(dfMDD$imprv1[responder==1], dfMDD$imprv1[responder==0])
        
        #t value
        table1[i,4] = round(t_test_res$statistic, 2)
    
        #p-value
        table1[i,5] = ifelse(t_test_res$p.value < 0.001, '<.001', round(t_test_res$p.value, 3))

        #Cohen's d
        table1[i,6] = round(cohen.d(dfMDD$imprv1[responder==1], dfMDD$imprv1[responder==0])$estimate, 2)
    }  
    
    
    #Responder PGI %
    table1[i,7] = paste0(round(sum(dfMDD[responder==1,]$imprv1 >= 3)/nrow(dfMDD[responder==1,])*100,1),'% (',
           sum(responder==1),')')
    
    #Non-Responder PGI%
    table1[i,8] = paste0(round(sum(dfMDD[responder==0,]$imprv1 >= 3)/nrow(dfMDD[responder==0,])*100,1),'% (',
           sum(responder==0),')')
    
    #Odds ratio
    odds_ratio_data = table(responder, dfMDD$imprv1 >= 3)
    if (all(odds_ratio_data>0)){
        odds_ratio_res = oddsratio(odds_ratio_data)
        odds_ratio_estimate = round(odds_ratio_res$measure[2,1],1)
        odds_ratio_lower = round(odds_ratio_res$measure[2,2],1)
        odds_ratio_upper = round(odds_ratio_res$measure[2,3],1)
        table1[i,9] = paste0(odds_ratio_estimate, ' (',odds_ratio_lower,'-',odds_ratio_upper,')')
    }
    
    
}
    


In [201]:
table1

38.7% (327),3.4 (0.7),2.4 (1.04),15.92,<.001,1.03,91.4% (327),52.2% (517),9.7 (6.4-15.1)
61.4% (518),3.2 (0.8),2.2 (1.05),15.22,<.001,1.15,84% (518),41.1% (326),7.5 (5.4-10.4)
36.1% (305),3.4 (0.7),2.4 (1.02),15.95,<.001,1.03,91.5% (305),53.8% (539),9.1 (6-14.5)
48.1% (406),3.3 (0.7),2.3 (0.99),17.55,<.001,1.2,89.9% (406),46.6% (438),10.2 (7.1-14.9)
30.3% (256),3.5 (0.6),2.5 (1.01),17.23,<.001,1.09,94.5% (256),55.6% (588),13.6 (8-25.1)
35.7% (301),3.4 (0.7),2.4 (0.99),17.67,<.001,1.15,94% (301),52.7% (543),14 (8.7-24)
25.1% (212),3.6 (0.6),2.5 (1),17.71,<.001,1.12,96.7% (212),57.6% (632),21.1 (10.5-50.4)
27.7% (234),3.5 (0.7),2.5 (0.99),16.69,<.001,1.1,96.2% (234),56.4% (610),19 (10.1-40.7)
20.4% (172),3.6 (0.6),2.6 (1),18.47,<.001,1.16,98.3% (172),59.5% (672),36.4 (13.6-153.1)
20.9% (176),3.6 (0.6),2.6 (1),18.47,<.001,1.15,98.3% (176),59.3% (668),37.6 (14.1-158.3)
11.8% (100),3.7 (0.4),2.7 (1.01),18.47,<.001,1.11,100% (100),63% (744),NA


In [196]:
table1 = matrix(NA, nrow = 11, ncol = 9)

for (i in 1:length(groups)){
    group = groups[i]
    responder = get(paste0(group,'_responder'))
    
    #N
    table1[i,1] = sum(responder)
    
    #Mean (SD)
    table1[i,2] = paste0(round(mean(dfMDD$imprv1[responder==1]),1), ' (', round(sd(dfMDD$imprv1[responder==1]),1),')')
    table1[i,3] = paste0(round(mean(dfMDD$imprv1[responder==0]),1), ' (', round(sd(dfMDD$imprv1[responder==0]),2),')')
    
    
    if (sum(responder) >1){
        #T-test
        t_test_res = t.test(dfMDD$imprv1[responder==1], dfMDD$imprv1[responder==0])
        
        #t value
        table1[i,4] = round(t_test_res$statistic, 2)
    
        #p-value
        table1[i,5] = ifelse(t_test_res$p.value < 0.001, '<.001', round(t_test_res$p.value, 3))

        #Cohen's d
        table1[i,6] = round(cohen.d(dfMDD$imprv1[responder==1], dfMDD$imprv1[responder==0])$estimate, 2)
    }  
    
    
    #Responder PGI %
    table1[i,7] = paste0(round(sum(dfMDD[responder==1,]$imprv1 >= 4)/nrow(dfMDD[responder==1,])*100,1),'% (',
           sum(responder==1),')')
    
    #Non-Responder PGI%
    table1[i,8] = paste0(round(sum(dfMDD[responder==0,]$imprv1 >= 4)/nrow(dfMDD[responder==0,])*100,1),'% (',
           sum(responder==0),')')
    
    #Odds ratio
    odds_ratio_data = table(responder, dfMDD$imprv1 >= 4)
    if (all(odds_ratio_data>0)){
        odds_ratio_res = oddsratio(odds_ratio_data)
        odds_ratio_estimate = round(odds_ratio_res$measure[2,1],1)
        odds_ratio_lower = round(odds_ratio_res$measure[2,2],1)
        odds_ratio_upper = round(odds_ratio_res$measure[2,3],1)
        table1[i,9] = paste0(odds_ratio_estimate, ' (',odds_ratio_lower,'-',odds_ratio_upper,')')
    }
    
    
}
    


In [197]:
table1

327,3.4 (0.7),2.4 (1.04),15.92,<.001,1.03,45.9% (327),13% (517),5.7 (4.1-8)
518,3.2 (0.8),2.2 (1.05),15.22,<.001,1.15,37.3% (518),7.4% (326),7.4 (4.8-11.9)
305,3.4 (0.7),2.4 (1.02),15.95,<.001,1.03,48.5% (305),12.8% (539),6.4 (4.6-9)
406,3.3 (0.7),2.3 (0.99),17.55,<.001,1.2,45.6% (406),7.3% (438),10.6 (7.1-16.1)
256,3.5 (0.6),2.5 (1.01),17.23,<.001,1.09,54.7% (256),13.1% (588),8 (5.7-11.3)
301,3.4 (0.7),2.4 (0.99),17.67,<.001,1.15,53.2% (301),10.5% (543),9.6 (6.8-13.8)
212,3.6 (0.6),2.5 (1),17.71,<.001,1.12,60.4% (212),14.1% (632),9.3 (6.5-13.3)
234,3.5 (0.7),2.5 (0.99),16.69,<.001,1.1,58.1% (234),13.3% (610),9 (6.4-12.9)
172,3.6 (0.6),2.6 (1),18.47,<.001,1.16,66.9% (172),15.2% (672),11.2 (7.7-16.5)
176,3.6 (0.6),2.6 (1),18.47,<.001,1.15,65.9% (176),15.1% (668),10.8 (7.4-15.8)
100,3.7 (0.4),2.7 (1.01),18.47,<.001,1.11,73% (100),19.4% (744),11.2 (7-18.3)


In [69]:
table(responder, dfMDD$imprv1 >= 3)

         
responder FALSE TRUE
        0   247  270
        1    28  299

In [58]:
i = 1
group = groups[i]
responder = get(paste0(group,'_responder'))
length(dfMDD$imprv1[responder==1])
t_test_res =  t.test(dfMDD$imprv1[responder==1], dfMDD$imprv1[responder==0])
t_test_res$statistic

cohen.d(dfMDD$imprv1[responder==1], dfMDD$imprv1[responder==0])
paste0(round(mean(dfMDD$imprv1[responder==1]),1), '(', round(sd(dfMDD$imprv1[responder==1]),1),')')

[1] 327

t 
15.92396


Cohen's d

d estimate: 1.027495 (large)
95 percent confidence interval:
   lower    upper 
0.880381 1.174610 


[1] "3.4(0.7)"

In [52]:
ifelse(t_test_res$p.value < 0.001, '<.001', round(t_test_res$p.value, 3))

[1] "<.001"